In [2]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import pandas as pd
import qiime2 as q2
from skbio import OrdinationResults
from qiime2 import Visualization
from seaborn import scatterplot
import matplotlib.pyplot as plt
import seaborn as sns
import biom
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

%matplotlib inline

In [3]:
#all variables
Data_raw='Data/raw'
Data_classified='Data/classified'
Data_diversity='Data/diversity'
Data_aroma='Data/aroma'

<div style="background-color: skyblue; padding: 10px;">
    Titles
    </div>
<div style="background-color: aliceblue; padding: 10px;">
    Results

Export all necessary files

In [8]:
!qiime tools export \
  --input-path $Data_classified/taxonomy.qza \
  --output-path $Data_aroma

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported Data/classified/taxonomy.qza as TSVTaxonomyDirectoryFormat to directory Data/aroma/exported-taxonomy


In [4]:
taxonomy = pd.read_csv("Data/aroma/taxonomy.tsv", sep="\t")

In [13]:
!qiime tools export \
  --input-path $Data_classified/table-filtered-sourdough_only.qza \
  --output-path $Data_aroma/table-filtered-sourdough_only

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported Data/classified/table-filtered-sourdough_only.qza as BIOMV210DirFmt to directory Data/aroma/table-filtered-sourdough_only


In [20]:
# Load BIOM file
table = biom.load_table('Data/aroma/table-filtered-sourdough_only/feature-table.biom')

# Convert to pandas DataFrame
table_filtered_sourdough = table.to_dataframe()

# Transpose so samples are rows, features are columns (because metadata file like this)
table_filtered_sourdough = table_filtered_sourdough.T


# Check dimensions
print(f"Shape: {table_filtered_sourdough.shape}")
print(f"First 5 samples:\n{table_filtered_sourdough.index[:5]}")
print(f"First 5 features:\n{table_filtered_sourdough.columns[:5]}")

Shape: (125, 1148)
First 5 samples:
Index(['366291_386-LP4-ITS-0386', '366291_387-LP4-ITS-0387',
       '366291_388-LP4-ITS-0388', '366291_389-LP4-ITS-0389',
       '366291_390-LP4-ITS-0390'],
      dtype='object')
First 5 features:
Index(['e275b97cfd9f38013f8b2fa887d236b5', 'cb41694c920fec98251cddcc554a3b34',
       '3a273034905d97733118d2aa8f7932c9', 'e0d94a06e608b36b4eb1f83bc0e5ce1c',
       'c12c4f383cf7839f434453e7361dbf68'],
      dtype='object')


In [ ]:
# Save to TSV
table_filtered_sourdough.to_csv('Data/aroma/table-filtered/feature-table-proper.tsv', sep='\t')

Prepare data

In [6]:
metadata = pd.read_csv("Data/raw/merged_output_usable.tsv", sep="\t") #using merged_output_usable because the spaces already substitued with _

aroma_columns = metadata.columns[-222:]

print(aroma_columns)

#so last 222 columns are aroma associated

Index(['ALCOHOLIC_D7', 'ANIMAL_FEED_D7', 'ANIMAL_STABLE_D7', 'APPLE_D7',
       'BANANA_D7', 'BEER_D7', 'BERRIES_D7', 'BREAD_D7', 'BUTTER_MILK_D7',
       'BUTYRIC_ACID_D7',
       ...
       'animal_score_D28', 'chemical_score_D28', 'body_odour_score_D28',
       'fruity_score_D28', 'maillard_score_D28', 'sour_score_D28',
       'ocean_score_D28', 'earthy_score_D28', 'fermented_dairy_score_D28',
       'nutty_score_D28'],
      dtype='object', length=222)


In [7]:
print("Now aroma df:", aroma_columns.shape, "Now metadata df:", metadata.shape)

Now aroma df: (222,) Now metadata df: (550, 304)


In [8]:
metadata_sd = metadata[metadata['sample_type'] == 'sourdough'].copy()

In [9]:
metadata_sd.head()
print("Now:", metadata_sd.shape)

Now: (125, 304)


In [10]:
# Check for missing values in aroma data
missing_aromas = metadata_sd[aroma_columns].isnull().sum()
if missing_aromas.any():
    print(f"\n⚠ Warning: Missing aroma values detected:")
    print(missing_aromas[missing_aromas > 0])


⚠ Warning: Missing aroma values detected:
PORRIDGE_D28                 21
WHOLE_GRAIN_D28              21
HAY_D28                      21
BREAD_D28                    21
CORN_D28                     21
                             ..
sour_score_D28               21
ocean_score_D28              21
earthy_score_D28             21
fermented_dairy_score_D28    21
nutty_score_D28              21
Length: 87, dtype: int64


In [11]:
metadata_sd[aroma_columns].isnull().sum(axis=1)

0       0
1       0
2       0
3       0
4      87
       ..
129     0
130     0
131     0
132    87
133     0
Length: 125, dtype: int64

In [12]:
print("Before:", metadata_sd.shape)
metadata_sd = metadata_sd.dropna(subset=aroma_columns, how='all')
print("After:", metadata_sd.shape)

Before: (125, 304)
After: (125, 304)


so there are no samples that miss all aroma analyses

In [13]:
# Extract genus from taxonomy
taxonomy['Genus'] = taxonomy['Taxon'].str.extract(r'g__([^;]+)')
taxonomy['Genus'] = taxonomy['Genus'].fillna('Unassigned')

print(f"\n✓ Extracted {taxonomy['Genus'].nunique()} unique genera")


✓ Extracted 1542 unique genera


In [29]:
# 1) Make sure taxonomy index is feature IDs
if 'Feature ID' in taxonomy.columns:
    taxonomy = taxonomy.set_index('Feature ID')

# 2) Extract genus
taxonomy['Genus'] = taxonomy['Taxon'].str.extract(r'g__([^;]+)')
taxonomy['Genus'] = taxonomy['Genus'].fillna('Unassigned')


# 3) Map ASVs to genus
feature_to_genus = taxonomy['Genus'].to_dict()

feature_table_genus = table_filtered_sourdough.copy()
feature_table_genus.columns = [
    feature_to_genus.get(col, 'Unknown') for col in feature_table_genus.columns
]

# 4) Collapse to genus
genus_table = feature_table_genus.groupby(level=0, axis=1).sum()

print("✓ Collapsed to genus level:", genus_table.shape[1], "genera")


/tmp/ipykernel_70/2546118294.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  genus_table = feature_table_genus.groupby(level=0, axis=1).sum()


✓ Collapsed to genus level: 253 genera


In [38]:
print("Genera:", genus_table.shape[1])
top_genera = genus_table.mean().sort_values(ascending=False).head(10)
print("\nTop 10 genera :")
for genus, abundance in top_genera.items():
    print(f"  {genus}: {abundance*100:.4f}%")


Genera: 253

Top 10 genera :
  Saccharomyces: 2078482.4000%
  Alternaria: 271887.2000%
  Triticum: 153444.0000%
  Unassigned: 109028.0000%
  Pichia: 39728.8000%
  Pyrenophora: 29515.2000%
  Cladosporium: 20733.6000%
  Aureobasidium: 20399.2000%
  Elymus: 16469.6000%
  Stemphylium: 7602.4000%


<div style="background-color: aliceblue; padding: 10px;">
Top 10 genera :
  Saccharomyces: 2078482.4000%  
  Alternaria: 271887.2000%  
  Triticum: 153444.0000%  
  Unassigned: 109028.0000%  
  Pichia: 39728.8000%  
  Pyrenophora: 29515.2000%  
  Cladosporium: 20733.6000%  
  Aureobasidium: 20399.2000%  
  Elymus: 16469.6000%  
  Stemphylium: 7602.4000%  

Merging metadata & genus

In [48]:
metadata = metadata.set_index('sample ID')

In [54]:
# Merge genus data with metadata
merged_data = genus_table.join(metadata, how='inner')

print(f"✓ Merged data: {merged_data.shape}")

# Define fungal columns (from genus table)
fungal_columns = genus_table.columns.tolist()

print(f"\nFinal dataset:")
print(f"  Total samples: {len(merged_data)}")
print(f"  Fungal genera: {len(fungal_columns)}")
print(f"  Aroma attributes: {len(aroma_columns)}")
print(f"  Timepoints: {sorted(merged_data['day'].unique())}")
print(f"  Backgrounds: {merged_data['background'].unique().tolist()}")

✓ Merged data: (125, 556)

Final dataset:
  Total samples: 125
  Fungal genera: 253
  Aroma attributes: 222
  Timepoints: [7.0, 14.0, 21.0]
  Backgrounds: ['non-sterile', 'sterile']


In [55]:
fungal_columns = list(fungal_columns)
aroma_columns = list(aroma_columns)

complete_cases = merged_data[fungal_columns + aroma_columns].notna().all(axis=1).sum()
print(f"  Complete cases (no missing data): {complete_cases}/{len(merged_data)}")


  Complete cases (no missing data): 104/125


## Spearman correlation
<div style="background-color: skyblue; padding: 10px;">
